## Введение в компьютерное зрение (CV): калибровка камеры

Эта домашка была построена на базе туториала OpenCV: https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

Что почитать:
* Конушин А. Геометрические свойства нескольких изображений: https://masters.donntu.ru/2020/etf/azin/library/article4.htm
* Примеры дисторсии и дугих искажений: https://help.autodesk.com/view/ARNOL/ENU/?guid=arnold_user_guide_ac_cameras_ac_perspective_camera_html

![img](https://help.autodesk.com/cloudhelp/ENU/AR-Core/images/ac-perspective-camera-radial-distortion-man-2-3.gif)
![img](https://upload.wikimedia.org/wikipedia/commons/d/d3/Camera_focal_length_distance_house_animation.gif)

Любая камера, которая используется для работы не идеальна. Матрица смещена относительно центра линзы, линза кривая, FOV настолько большой, что у людей на катринках носы больше головы. Чтобы компенсировать искажения, людьми были придуманы некоторые алгоритмы. Но сначала теория:

### 1. Немного теории
Презентации: 
* (стр. 1-10) http://www.myshared.ru/slide/1351161/
* https://3d.bk.tudelft.nl/courses/geo1016/slides/Lecture_03_Calibration.pdf

Радиальная дисторсия be like:

![distortion](https://docs.opencv.org/4.x/distortion_examples.png)

Для лучшего понимания математического описания дисторсии стоит прочиать [basics из туториала OpenCV](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html).

Примеры калибровочных шаблонов:

![templates](https://www.researchgate.net/profile/Andrew-Hieu-Nguyen/publication/323782425/figure/fig3/AS:1112642001350662@1642285933669/Example-of-four-commonly-used-camera-calibration-targets.png)

### 2. Практика


1) Теоретическая задачка на разогрев. Определите параметры (диапазон значений) матрицы внутренней калибровки для камеры Panasonic DMC-FZ20 (**1 point**):

https://www.dpreview.com/products/panasonic/compacts/panasonic_dmcfz20/specifications


In [2]:
# %pip install matplotlib==3.8.2 numpy==1.26.3 opencv-python==4.7.0.68

Ниже ячейка, где вы можете сфотографировать свои шаблоны (`patterns/pattern.png`). Фото делается по пробелу, выход из программы - escape. Фотографии сохраняются в папку `images/`

In [3]:
import take_photos

take_photos.main(0) # Input: number of camera

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/kirill/Robotics/wheeled_robots_intro/hw_4/.venv/lib/python3.12/site-packages/cv2/qt/plugins"


Taken images were saved in './images/'


Найдем углы квадратов шаблона:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import glob
import os

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((9*6, 3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('test_images/*.png')

plt.figure(figsize=(15, 60))

for i, fname in enumerate(images):
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (9, 6), None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (9, 6), corners2, ret)

    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.subplot(int(np.ceil(len(images) / 2)), 2, i + 1)
    plt.imshow(img)

plt.show()

Получаем матрицу камеры, коэффициенты дисторсии, векторы поворотов и положений камеры:

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print(mtx, dist, rvecs, tvecs, sep='\n\n')

2) Прокомментируйте полученные значения матрицы внутренней калибровки. Какие физические параметры камеры можно сказать, а какие нельзя? (**1 point**)

А что там с дисторсией?

![dist](https://arnold-rendering.com/wp-content/uploads/2017/06/radial_distortion_face_user-guide.gif?w=600)

Уточним матрицу камеры:

In [ ]:
img = cv.imread('test_images/img5.png')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
print(newcameramtx, roi, sep='\n\n')

Избавимся от дисторсии и нарисуем, что получилось:

In [ ]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi
cropped = dst[y:y+h, x:x+w]

img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
dst = cv.cvtColor(dst, cv.COLOR_BGR2RGB)
cropped = cv.cvtColor(cropped, cv.COLOR_BGR2RGB)

plt.figure(figsize=(10, 20))

plt.subplot(3, 1, 1)
plt.imshow(img)
plt.subplot(3, 1, 2)
plt.imshow(dst)
plt.subplot(3, 1, 3)
plt.imshow(cropped)
plt.show()

Еще примеры:

![before](https://i.stack.imgur.com/xNc5k.jpg)
![after](https://i.stack.imgur.com/O9osX.jpg)

Посчитаем ошибку репроекции:

In [ ]:
mean_error = 0

for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2) / len(imgpoints2)
    mean_error += error

total_error = mean_error / len(objpoints)
print( "total error: {}".format(total_error))

3) За ошибку репроекции < 0.2 дается **1 point**

In [ ]:
assert total_error < 0.2

In [ ]:
img_index = 5

img = cv.imread(images[img_index])

imgpoints2, _ = cv.projectPoints(objpoints[img_index], rvecs[img_index], tvecs[img_index], mtx, dist)
cur_imgpoints = np.array(imgpoints[img_index])

for i in range(imgpoints2.shape[0]):
    img = cv.circle(img, cur_imgpoints[i, 0].astype('int'), 1, (0, 0, 255), 1)
    img = cv.circle(img, imgpoints2[i, 0].astype('int'), 1, (255, 0, 0), 1)

img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
plt.figure(figsize=(20, 40))
plt.imshow(img)
plt.show()

4) Определите размер ячейки шахматной доски, листа А4 или любого другого объекта, если знаете расстояние до него. Сделаете замер расстояния при фотографировании (**2 points**)

In [ ]:
# TODO
# 28 см

### Extra задание: сделайте калибровку камеры по шаблону с кругами (**2 points**)

В этом поможет: `cv2.findСirclesGrid` [[link]](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#gac8fb88c7652a7699743aeb6073ef9b4a). Так же вы можете пользоваться и другими библиотеками, например [skimage](https://scikit-image.org/).

Паттерн калибровки лежит в `patterns/circles_pattern.jpg`